**1.** Download required files (first-order-model, template, audio and the actual model files).

In [ ]:
!cd /content/ && git clone --quiet https://github.com/AliaksandrSiarohin/first-order-model & git clone --quiet https://github.com/Weilbyte/bakamitai_deepfake
!cd /content/bakamitai_deepfake/ && wget -nc https://github.com/Weilbyte/bakamitai_deepfake/releases/download/model/vox-cpk.pth.tar -q --show-progress
!cd /content/bakamitai_deepfake/ && wget -nc https://github.com/Weilbyte/bakamitai_deepfake/releases/download/model/vox-adv-cpk.pth.tar -q --show-progress

**2.** Load the model.

In [ ]:
from os import chdir
chdir('/content/first-order-model')
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/content/bakamitai_deepfake/vox-cpk.pth.tar')

**3.** Upload source image.

In [ ]:
from os import chdir
chdir('/content/')

# Clean-up ;)
from os import remove
try:
  remove(f'/content/{uploaded_name}')
  remove(f'/content/{uploaded_name}.mp4')
except:
  pass

# Upload 
from os import rename
from google.colab import files
uploaded = files.upload()
uploaded_name = list(uploaded.keys())[0]

# Read 
from imageio import imread, mimread
template = mimread('/content/bakamitai_deepfake/template.mp4')
uploaded_image = imread(f'/content/{uploaded_name}')

# Resize
from skimage.transform import resize
template = [resize(frame, (256, 256))[..., :3] for frame in template]
uploaded_image = resize(uploaded_image, (256, 256))[..., :3]


# Deepfake
chdir('first-order-model')
from imageio import mimsave
from demo import make_animation
from skimage import img_as_ubyte
import warnings
warnings.filterwarnings("ignore")

predictions = make_animation(uploaded_image, template, generator, kp_detector, relative=True)
mimsave(f'/content/{uploaded_name}.mp4', [img_as_ubyte(frame) for frame in predictions])

# Audio
from moviepy.editor import VideoFileClip, vfx, AudioFileClip, CompositeAudioClip
video = VideoFileClip(f'/content/{uploaded_name}.mp4')
audio = AudioFileClip('/content/bakamitai_deepfake/audio.mp3')
final_video = video.fx(vfx.speedx, 3)
final_video = final_video.set_audio(audio)
remove(f'/content/{uploaded_name}.mp4')
final_video.write_videofile(f'/content/{uploaded_name}.mp4')

# Download
files.download(f'/content/{uploaded_name}.mp4')